# Electricity Demand & Weather Proprecessing

This notebook explains step by step how the preprocessing pipeline works to build a clean, continuous, hourly dataset combining:

- Electricity demand (load_MW)
- Weather variables

# 1. Context & Objectives 

We start from raw time series data that may contain:
- Missing timestamps
- Irregular hourly frequency
- Gaps (NaNs)

### Final objectives: 
- Ensure a complete hourly timeline
- Interpolate numeric variables only
- Safely handle categorical variables
- Merge demand and weather into a single dataset
- Produce a model‑ready dataset

# 2. Imports & Project Structure

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# We rely on pathlib for robust and portable file paths
PROJECT_ROOT = Path.cwd().parents[0]
RAW_BASE_PATH = PROJECT_ROOT / "data" / "raw"
PROCESSED_BASE_PATH = PROJECT_ROOT / "data" / "processed"

# 3. Load Raw Data

In [3]:
country = "FR"
year = 2023

# Raw file paths
demand_path = (
RAW_BASE_PATH
/ "electricity_demand"
/ f"country={country}"
/ f"year={year}"
/ "demand.parquet"
)

weather_path = (
RAW_BASE_PATH
/ "weather"
/ f"country={country}"
/ f"year={year}"
/ "weather.parquet"
)

# Load dataframes
df_demand = pd.read_parquet(demand_path)
df_weather = pd.read_parquet(weather_path)

In [4]:
df_demand.head()

,datetime,load_MW,country
0,2023-01-01 00:00:00+00:00,45709.0,FR
1,2023-01-01 01:00:00+00:00,44640.0,FR
2,2023-01-01 02:00:00+00:00,41533.0,FR
3,2023-01-01 03:00:00+00:00,39248.0,FR
4,2023-01-01 04:00:00+00:00,38389.0,FR


In [5]:
df_weather.head()

,datetime,temperature_2m,relative_humidity_2m,wind_speed_10m,shortwave_radiation_instant,country
0,2023-01-01 00:00:00+00:00,14.85,53.719143,27.859905,0.0,FR
1,2023-01-01 01:00:00+00:00,14.95,52.638969,26.302181,0.0,FR
2,2023-01-01 02:00:00+00:00,14.75,53.321426,23.065300,0.0,FR
3,2023-01-01 03:00:00+00:00,14.20,55.827904,21.385939,0.0,FR
4,2023-01-01 04:00:00+00:00,14.15,57.980919,20.683559,0.0,FR


# 4. Helper functions

In [6]:
# Build a full hourly time index
def build_full_hourly_index(df: pd.DataFrame, time_col: str) -> pd.DatetimeIndex:
    """
    Build a complete hourly DatetimeIndex between min and max timestamps.
    """
    return pd.date_range(
    start=df[time_col].min(),
    end=df[time_col].max(),
    freq="h"
)

Why this matters?
- Models expect regular time steps,
- Missing hours must be explicitly created before interpolation

In [7]:
# Reindex & interpolate a time series
def reindex_and_interpolate_ts(
    df: pd.DataFrame,
    time_col: str,
    numeric_cols: list[str],
    categorical_cols: list[str] | None = None,
) -> pd.DataFrame:

    df = (
        df
        .drop_duplicates(subset=time_col)
        .sort_values(time_col)
        .copy()
    )

    df[time_col] = pd.to_datetime(df[time_col])

    full_index = build_full_hourly_index(df, time_col)

    df = (
        df
        .set_index(time_col)
        .reindex(full_index)
    )

    df[numeric_cols] = (
        df[numeric_cols]
        .interpolate(method="time", limit_area="inside")
    )

    if categorical_cols:
        df[categorical_cols] = (
            df[categorical_cols]
            .ffill()
            .bfill()
        )

    df = (
        df
        .rename_axis(time_col)
        .reset_index()
    )

    assert df[time_col].is_monotonic_increasing

    return df

# 5. Functions application

## 5.1. Demand data

In [8]:
df_demand_processed = reindex_and_interpolate_ts(
df=df_demand,
time_col="datetime",
numeric_cols=["load_MW"],
categorical_cols=["country"]
)

In [9]:
df_demand_processed.head()

,datetime,load_MW,country
0,2023-01-01 00:00:00+00:00,45709.0,FR
1,2023-01-01 01:00:00+00:00,44640.0,FR
2,2023-01-01 02:00:00+00:00,41533.0,FR
3,2023-01-01 03:00:00+00:00,39248.0,FR
4,2023-01-01 04:00:00+00:00,38389.0,FR


In [10]:
df_demand_processed.isna().sum()

datetime    0
load_MW     0
country     0
dtype: int64

## 5.2. Weather data

In [11]:
weather_cols = [
"temperature_2m",
"relative_humidity_2m",
"wind_speed_10m",
"shortwave_radiation_instant"
]

df_weather_processed = reindex_and_interpolate_ts(
df=df_weather,
time_col="datetime",
numeric_cols=weather_cols
)

In [12]:
df_weather_processed.head()

,datetime,temperature_2m,relative_humidity_2m,wind_speed_10m,shortwave_radiation_instant,country
0,2023-01-01 00:00:00+00:00,14.85,53.719143,27.859905,0.0,FR
1,2023-01-01 01:00:00+00:00,14.95,52.638969,26.302181,0.0,FR
2,2023-01-01 02:00:00+00:00,14.75,53.321426,23.065300,0.0,FR
3,2023-01-01 03:00:00+00:00,14.20,55.827904,21.385939,0.0,FR
4,2023-01-01 04:00:00+00:00,14.15,57.980919,20.683559,0.0,FR


# 6. Merge Demand & Weather DataFrames

In [13]:
# Weather: drop redundant metadata
df_weather_processed = df_weather_processed.drop(columns=["country"])

df_merged = df_demand_processed.merge(
df_weather_processed,
on="datetime",
how="inner"
)

# Add metadata
df_merged["year"] = year

In [14]:
df_merged.head()

,datetime,load_MW,country,temperature_2m,relative_humidity_2m,wind_speed_10m,shortwave_radiation_instant,year
0,2023-01-01 00:00:00+00:00,45709.0,FR,14.85,53.719143,27.859905,0.0,2023
1,2023-01-01 01:00:00+00:00,44640.0,FR,14.95,52.638969,26.302181,0.0,2023
2,2023-01-01 02:00:00+00:00,41533.0,FR,14.75,53.321426,23.065300,0.0,2023
3,2023-01-01 03:00:00+00:00,39248.0,FR,14.20,55.827904,21.385939,0.0,2023
4,2023-01-01 04:00:00+00:00,38389.0,FR,14.15,57.980919,20.683559,0.0,2023


In [15]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   datetime                     8760 non-null   datetime64[ns, UTC]
 1   load_MW                      8760 non-null   float64            
 2   country                      8760 non-null   object             
 3   temperature_2m               8760 non-null   float32            
 4   relative_humidity_2m         8760 non-null   float32            
 5   wind_speed_10m               8760 non-null   float32            
 6   shortwave_radiation_instant  8760 non-null   float32            
 7   year                         8760 non-null   int64              
dtypes: datetime64[ns, UTC](1), float32(4), float64(1), int64(1), object(1)
memory usage: 410.8+ KB


# 7. Final Data Quality Checks

In [16]:
# No missing values
assert df_merged.isna().sum().sum() == 0

# Strict hourly continuity
assert df_merged['datetime'].diff().dropna().unique()[0] == pd.Timedelta('1h')

The dataset is now clean, regular and model-ready.

This preprocessing logic is reused as-is inside the production script src/preprocessing/build_preprocessed_dataset.py.